In [1]:
import pandas as pd
import numpy as np
import random
import sys
import os
import scipy as sc
import warnings
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'

#############################################################################################
######################  Load Data   #########################################################
#############################################################################################

main_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-RAND/Compiled_HCRIS-HACRP-RAND.pkl')
print(main_df.shape)
main_df = main_df[main_df['STATE'] != 'MD']
print(main_df.shape)
main_df.head()

(26218, 56)
(26123, 56)


,RPT_REC_NUM,S2_1_C1_35,S2_1_C1_40,S2_1_C2_40,IPPS interim payment (E_A_HOS_C1_72),E_A_HOS_C1_7096,E_A_HOS_C1_7097,E_A_HOS_C1_7093,E_A_HOS_C1_7090,E_A_HOS_C1_7094,E_A_HOS_C1_7091,E_A_HOS_C1_49,E_A_HOS_C1_50,E_A_HOS_C1_54,IPPS payment (E_A_HOS_C1_59),E_A_HOS_C1_68,E_A_HOS_C1_93,E_A_HOS_C1_47,E_A_HOS_C1_48,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,NPI,RPT_STUS_CD,FY_BGN_DT,FY_END_DT,PROC_DT,FILE_YEAR,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),file_month,Payment Reduction,Payment Reduction Footnote,Total HAC Score,Total HAC Footnote,HAI Measures Start Date,HAI Measures End Date,Total device days,HAC penalty (E_A_HOS_C1_7099),CAUTI Urinary Catheter Days,CLABSI Number of Device Days,MRSA patient days,CDI patient days,HAC penalty (imputed from RAND),STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
3169,699112,1.0,Y,N,76191.0,874.0,14695.0,NaN,NaN,-50.0,NaN,57819.0,4454.0,NaN,62273.0,NaN,NaN,57819.0,5749.0,2,450219,,2,09/01/2020,12/31/2020,09/29/2021,2020,62273.0,77792.0,778.0,77014.0,622.73,62272.98,10,Yes,NaN,1.3968,NaN,2017-01-01,2018-12-31,150.0,53.0,133.0,17.0,1398.0,1398.0,629.0,TX,0.02,0.000032,1086.792453,576.0,1367.924528,725.0,6230.333333,778.0
3184,645789,NaN,Y,N,327887.0,NaN,72394.0,NaN,NaN,-53.0,NaN,300956.0,21052.0,NaN,322008.0,NaN,NaN,300956.0,265783.0,4,370030,,2,09/03/2016,12/31/2016,04/17/2019,2016,322008.0,394349.0,3943.0,390406.0,3220.08,NaN,11,Yes,NaN,8.0,NaN,2013-01-01,2014-12-31,NaN,59.0,NaN,NaN,NaN,NaN,NaN,OK,NaN,NaN,NaN,NaN,6583.050847,3884.0,15242.875000,3943.0
3186,725518,NaN,Y,N,5475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,440231,,2,07/01/2018,06/30/2019,09/29/2022,2018,NaN,0.0,NaN,0.0,NaN,NaN,10,Yes,NaN,0.8814,NaN,2015-01-01,2016-12-31,773.0,63.0,327.0,446.0,13353.0,13353.0,NaN,TN,NaN,NaN,NaN,NaN,NaN,NaN,3363.750000,63.0
3187,632292,NaN,N,Y,4965.0,NaN,NaN,NaN,NaN,NaN,NaN,6979.0,563.0,NaN,7542.0,NaN,NaN,6979.0,NaN,10,490104,,2,07/01/2015,06/30/2016,09/17/2018,2015,7542.0,7542.0,75.0,7467.0,75.42,NaN,10,No,NaN,NaN,NaN,2012-01-01,2013-12-31,NaN,75.0,NaN,NaN,NaN,NaN,NaN,VA,NaN,NaN,NaN,NaN,0.000000,0.0,206.666667,75.0
3188,659081,NaN,Y,Y,7048.0,NaN,NaN,NaN,NaN,NaN,NaN,7428.0,608.0,NaN,8036.0,NaN,NaN,7428.0,NaN,10,490104,,2,07/01/2016,06/30/2017,11/22/2019,2016,8036.0,8036.0,80.0,7956.0,80.36,NaN,11,Yes,NaN,9.25,NaN,2013-01-01,2014-12-31,NaN,80.0,NaN,NaN,NaN,NaN,NaN,VA,NaN,NaN,NaN,NaN,0.000000,0.0,206.666667,80.0


## Before filtering out NaNs from 'HAC penalty, final'

In [2]:
#############################################################################################
##############  Separate into 2 dataframes penalized Y and penalized N  #####################
#############################################################################################

hac_df_y = main_df[main_df['Payment Reduction'] == 'Yes']
print('rows in hac_df_y:', hac_df_y.shape[0])

hac_df_n = main_df[main_df['Payment Reduction'] == 'No']
print('rows in hac_df_n:', hac_df_n.shape[0], '\n')


#############################################################################################
##############  Loop through years  #####################
#############################################################################################

years = ['2015', '2016', '2017', '2018', '2019',
         '2020', '2021', '2022']

Days_ap = []
Days_pp = []
Actual_Penalties = []
Potential_Penalties = []

for i, yr in enumerate(years):    
    print(yr)
    
    hac_tdf = hac_df_n[hac_df_n['FILE_YEAR'] == yr]
    hac_tdf.drop_duplicates(subset=['PRVDR_NUM'], inplace=True)
    hac_hosps = sorted(list(set(hac_tdf['PRVDR_NUM'].tolist())))
    print('   ', len(hac_hosps), 'non-penalized hospitals')
    hac_tdf = hac_tdf[~hac_tdf['HAC penalty, final'].isin([np.nan, float('NaN')])]
    hac_tdf = hac_tdf[~hac_tdf['Total device days'].isin([np.nan, float('NaN')])]
    penalties = hac_tdf['HAC penalty, final'].tolist()
    Potential_Penalties.append(penalties)
    Days_pp.append(hac_tdf['Total device days'].tolist())
    
    hac_tdf = hac_df_y[hac_df_y['FILE_YEAR'] == yr]
    hac_tdf.drop_duplicates(subset=['PRVDR_NUM'], inplace=True)
    hac_hosps1 = sorted(list(set(hac_tdf['PRVDR_NUM'].tolist())))
    print('   ', len(hac_hosps1), 'penalized hospitals')
    hac_tdf = hac_tdf[~hac_tdf['HAC penalty, final'].isin([np.nan, float('NaN')])]
    hac_tdf = hac_tdf[~hac_tdf['Total device days'].isin([np.nan, float('NaN')])]
    penalties = hac_tdf['HAC penalty, final'].tolist()
    Actual_Penalties.append(penalties)
    Days_ap.append(hac_tdf['Total device days'].tolist())
    
    print('   ', len(hac_hosps1) + len(hac_hosps), 'hospitals in the HACRP')
    print('   ', np.round(100*len(hac_hosps1)/(len(hac_hosps1) + len(hac_hosps)),1), '% were penalized')
    tpen = np.nansum(penalties)
    print('   ', '${0}'.format(format(tpen, ',.2f')), 'total penalties')
    

rows in hac_df_y: 6445
rows in hac_df_n: 19678 

2015
    2443 non-penalized hospitals
    847 penalized hospitals
    3290 hospitals in the HACRP
    25.7 % were penalized
    $389,213,213.00 total penalties
2016
    2425 non-penalized hospitals
    880 penalized hospitals
    3305 hospitals in the HACRP
    26.6 % were penalized
    $474,560,217.57 total penalties
2017
    2499 non-penalized hospitals
    768 penalized hospitals
    3267 hospitals in the HACRP
    23.5 % were penalized
    $453,468,246.19 total penalties
2018
    2509 non-penalized hospitals
    749 penalized hospitals
    3258 hospitals in the HACRP
    23.0 % were penalized
    $328,574,413.35 total penalties
2019
    2434 non-penalized hospitals
    800 penalized hospitals
    3234 hospitals in the HACRP
    24.7 % were penalized
    $359,674,470.27 total penalties
2020
    2391 non-penalized hospitals
    786 penalized hospitals
    3177 hospitals in the HACRP
    24.7 % were penalized
    $370,056,849.07 total p

## After filtering out NaNs from 'HAC penalty, final'

In [3]:
main_df = main_df[~main_df['HAC penalty, final'].isin([np.nan, float('NaN')])]

#############################################################################################
##############  Separate into 2 dataframes penalized Y and penalized N  #####################
#############################################################################################

hac_df_y = main_df[main_df['Payment Reduction'] == 'Yes']
print('rows in hac_df_y:', hac_df_y.shape[0])

hac_df_n = main_df[main_df['Payment Reduction'] == 'No']
print('rows in hac_df_n:', hac_df_n.shape[0], '\n')


#############################################################################################
##############  Loop through years  #####################
#############################################################################################

years = ['2015', '2016', '2017', '2018', '2019',
         '2020', '2021', '2022']

Days_ap = []
Days_pp = []
Actual_Penalties = []
Potential_Penalties = []

for i, yr in enumerate(years):    
    print(yr)
    
    hac_tdf = hac_df_n[hac_df_n['FILE_YEAR'] == yr]
    hac_tdf.drop_duplicates(subset=['PRVDR_NUM'], inplace=True)
    hac_hosps = sorted(list(set(hac_tdf['PRVDR_NUM'].tolist())))
    print('   ', len(hac_hosps), 'non-penalized hospitals')
    hac_tdf = hac_tdf[~hac_tdf['HAC penalty, final'].isin([np.nan, float('NaN')])]
    hac_tdf = hac_tdf[~hac_tdf['Total device days'].isin([np.nan, float('NaN')])]
    penalties = hac_tdf['HAC penalty, final'].tolist()
    Potential_Penalties.append(penalties)
    Days_pp.append(hac_tdf['Total device days'].tolist())
    
    hac_tdf = hac_df_y[hac_df_y['FILE_YEAR'] == yr]
    hac_tdf.drop_duplicates(subset=['PRVDR_NUM'], inplace=True)
    hac_hosps1 = sorted(list(set(hac_tdf['PRVDR_NUM'].tolist())))
    print('   ', len(hac_hosps1), 'penalized hospitals')
    hac_tdf = hac_tdf[~hac_tdf['HAC penalty, final'].isin([np.nan, float('NaN')])]
    hac_tdf = hac_tdf[~hac_tdf['Total device days'].isin([np.nan, float('NaN')])]
    penalties = hac_tdf['HAC penalty, final'].tolist()
    Actual_Penalties.append(penalties)
    Days_ap.append(hac_tdf['Total device days'].tolist())
    
    print('   ', len(hac_hosps1) + len(hac_hosps), 'hospitals in the HACRP')
    print('   ', np.round(100*len(hac_hosps1)/(len(hac_hosps1) + len(hac_hosps)),1), '% were penalized')
    tpen = np.nansum(penalties)
    print('   ', '${0}'.format(format(tpen, ',.2f')), 'total penalties')
    

rows in hac_df_y: 6384
rows in hac_df_n: 19544 

2015
    2421 non-penalized hospitals
    834 penalized hospitals
    3255 hospitals in the HACRP
    25.6 % were penalized
    $389,213,213.00 total penalties
2016
    2397 non-penalized hospitals
    873 penalized hospitals
    3270 hospitals in the HACRP
    26.7 % were penalized
    $474,560,217.57 total penalties
2017
    2491 non-penalized hospitals
    760 penalized hospitals
    3251 hospitals in the HACRP
    23.4 % were penalized
    $453,468,246.19 total penalties
2018
    2502 non-penalized hospitals
    739 penalized hospitals
    3241 hospitals in the HACRP
    22.8 % were penalized
    $328,574,413.35 total penalties
2019
    2427 non-penalized hospitals
    790 penalized hospitals
    3217 hospitals in the HACRP
    24.6 % were penalized
    $359,674,470.27 total penalties
2020
    2386 non-penalized hospitals
    779 penalized hospitals
    3165 hospitals in the HACRP
    24.6 % were penalized
    $370,056,849.07 total p

In [4]:
T_ia_pen_2022 = 78297872.94
p_ia_pen = 105/764

T_a_pen_2022 = 18070853.76
p_a_pen = 104/2296

############################################################################################
###############  Analyze penalties and generate output data  ###############################
############################################################################################

Avgs_ia = []
Stds_ia = []
T_lo_ia = 0
T_hi_ia = 0
T_avg_ia = 0

Avgs_a = []
Stds_a = []
T_lo_a = 0
T_hi_a = 0
T_avg_a = 0

Avgs_s = []
Stds_s = []
T_lo_s = 0
T_hi_s = 0
T_avg_s = 0

iterations = 10**4

for i, yr in enumerate(years):
    actual_penalties = Actual_Penalties[i]
    actual_penalties2 = []
    days_ap = Days_ap[i]
    days_ap2 = []
    
    potential_penalties = Potential_Penalties[i]
    potential_penalties2 = []
    days_pp = Days_pp[i]
    days_pp2 = []
    
    for j, ap in enumerate(actual_penalties):
        if ap > 0 and ap < 10**10:
            actual_penalties2.append(ap)
            days_ap2.append(days_ap[j])
            
    for j, pp in enumerate(potential_penalties):
        if pp > 0 and pp < 10**10:
            potential_penalties2.append(pp)
            days_pp2.append(days_pp[j])
            
    Ts_ia = []
    Ts_a = []
    Ts_s = []
    z = 0.49
    for i in range(iterations):
        
        n = np.random.uniform(0.13, 0.14)
        sz = int(np.round(n * len(actual_penalties2)))
        
        weights = np.array(days_ap2)**z
        weights = weights/np.sum(weights)
        
        p_set_ia = np.random.choice(actual_penalties2, size=sz, replace=False, p=weights)
        Ts_ia.append(np.nansum(p_set_ia))
        
        
        n = np.random.uniform(0.04, 0.05)
        sz = int(np.round(n * len(potential_penalties2)))
        
        weights = (1/np.array(days_pp2))**z
        weights = weights/np.sum(weights)
        
        p_set_a = np.random.choice(potential_penalties2, size=sz, replace=False, p=weights)
        Ts_a.append(np.nansum(p_set_a))
        
        Ts_s.append(np.nansum(p_set_ia) - np.nansum(p_set_a))
    
    
    
    avg_ia = np.nanmean(Ts_ia)
    std_ia = np.nanstd(Ts_ia)
    Avgs_ia.append(avg_ia)
    Stds_ia.append(std_ia)
    avg_M_ia = "${:,.2f}".format(np.round(avg_ia, 2))
    std_M_ia = "${:,.2f}".format(np.round(std_ia, 2))
    
    avg_a = np.nanmean(Ts_a)
    std_a = np.nanstd(Ts_a)
    Avgs_a.append(avg_a)
    Stds_a.append(std_a)
    avg_M_a = "${:,.2f}".format(np.round(avg_a, 2))
    std_M_a = "${:,.2f}".format(np.round(std_a, 2))
    
    avg_s = np.nanmean(Ts_s)
    std_s = np.nanstd(Ts_s)
    Avgs_s.append(avg_s)
    Stds_s.append(std_s)
    avg_M_s = "${:,.2f}".format(np.round(avg_s, 2))
    std_M_s = "${:,.2f}".format(np.round(std_s, 2))
    
    
    T_lo_ia += avg_ia - std_ia
    T_hi_ia += avg_ia + std_ia
    T_avg_ia += avg_ia
        
    T_lo_a += avg_a - std_a
    T_hi_a += avg_a + std_a
    T_avg_a += avg_a
        
    T_lo_s += avg_s - std_s
    T_hi_s += avg_s + std_s
    T_avg_s += avg_s
    
    print(yr, 'Avg sum of expected inappropriate penalties:', avg_M_ia, '±', std_M_ia, 'SD')
    print(yr, 'Avg sum of expected appropriate penalties:', avg_M_a, '±', std_M_a, 'SD')
    print(yr, 'Avg expected savings for CMS:', avg_M_s, '±', std_M_s, 'SD')
    print('\n')




2015 Avg sum of expected inappropriate penalties: $79,244,228.77 ± $6,334,529.52 SD
2015 Avg sum of expected appropriate penalties: $19,198,347.00 ± $2,952,809.20 SD
2015 Avg expected savings for CMS: $60,045,881.77 ± $6,994,151.91 SD


2016 Avg sum of expected inappropriate penalties: $97,983,424.35 ± $8,195,853.95 SD
2016 Avg sum of expected appropriate penalties: $17,488,580.61 ± $2,746,604.21 SD
2016 Avg expected savings for CMS: $80,494,843.74 ± $8,612,738.01 SD


2017 Avg sum of expected inappropriate penalties: $88,690,047.31 ± $7,419,980.26 SD
2017 Avg sum of expected appropriate penalties: $16,025,755.05 ± $2,756,986.91 SD
2017 Avg expected savings for CMS: $72,664,292.27 ± $7,912,908.12 SD


2018 Avg sum of expected inappropriate penalties: $69,856,635.48 ± $6,580,587.57 SD
2018 Avg sum of expected appropriate penalties: $18,286,213.88 ± $3,314,287.54 SD
2018 Avg expected savings for CMS: $51,570,421.60 ± $7,342,912.71 SD


2019 Avg sum of expected inappropriate penalties: $7

In [5]:
avg_T_ia = []
std_T_ia = []
avg_T_a = []
std_T_a = []
avg_T_s = []
std_T_s = []

z = 0.49

yr = 2015
yrs = []
for j in range(len(years)):
    
    yrs.append(yr)
    yr +=1
    
    Tsum_ia = []
    Tsum_a = []
    Tsum_s = []
    
    yr_subset = years[0:j+1]
    for i in range(iterations):
        ia_penalties_by_year = []
        a_penalties_by_year = []
        
        for ii, yr1 in enumerate(yr_subset):
            
            actual_penalties = Actual_Penalties[ii]
            actual_penalties2 = []
            days_ap = Days_ap[ii]
            days_ap2 = []
            
            for jj, ap in enumerate(actual_penalties):
                if ap > 0 and ap < 10**10:
                    actual_penalties2.append(ap)
                    days_ap2.append(days_ap[jj])
                    
            n = np.random.uniform(0.13, 0.14)
            sz = int(np.round(n * len(actual_penalties)))
            
            weights = np.array(days_ap2)**z
            weights = weights/np.sum(weights)
            
            p_set = np.random.choice(actual_penalties, size=sz, replace=False, p=weights)
            ia_penalties_by_year.extend(p_set)
            
            potential_penalties = Potential_Penalties[ii]
            potential_penalties2 = []
            days_pp = Days_pp[ii]
            days_pp2 = []
            
            for jj, pp in enumerate(potential_penalties):
                if pp > 0 and pp < 10**10:
                    potential_penalties2.append(pp)
                    days_pp2.append(days_pp[jj])
                    
            n = np.random.uniform(0.04, 0.05)
            sz = int(np.round(n * len(potential_penalties2)))
            
            weights = (1/np.array(days_pp2))**z
            weights = weights/np.sum(weights)
            
            p_set = np.random.choice(potential_penalties2, size=sz, replace=False, p=weights)
            a_penalties_by_year.extend(p_set)
            
        Tsum_ia.append(np.nansum(ia_penalties_by_year))
        Tsum_a.append(np.nansum(a_penalties_by_year))
        Tsum_s.append(np.nansum(ia_penalties_by_year) - np.nansum(a_penalties_by_year))

    avg_T_ia.append(np.nanmean(Tsum_ia))
    std_T_ia.append(np.nanstd(Tsum_ia))
    
    avg_T_a.append(np.nanmean(Tsum_a))
    std_T_a.append(np.nanstd(Tsum_a))
    
    avg_T_s.append(np.nanmean(Tsum_s))
    std_T_s.append(np.nanstd(Tsum_s))

print('Analyzing all years together:')
print(' Avg total sum of expected inappropriate penalties:', "${:,.2f}".format(np.round(avg_T_ia[-1], 2)), '±', "${:,.2f}".format(np.round(std_T_ia[-1], 2)), 'SD')
print(' Avg total sum of expected appropriate penalties:', "${:,.2f}".format(np.round(avg_T_a[-1], 2)), '±', "${:,.2f}".format(np.round(std_T_a[-1], 2)), 'SD')
print(' Avg total sum of expected CMS savings:', "${:,.2f}".format(np.round(avg_T_s[-1], 2)), '±', "${:,.2f}".format(np.round(std_T_s[-1], 2)), 'SD')
print('\n\n')



Analyzing all years together:
 Avg total sum of expected inappropriate penalties: $649,627,922.98 ± $20,684,991.64 SD
 Avg total sum of expected appropriate penalties: $141,096,195.46 ± $8,675,657.47 SD
 Avg total sum of expected CMS savings: $508,531,727.51 ± $22,324,476.03 SD





In [6]:
df = pd.DataFrame(columns=['year', 'cum_sum', 'cum_sum_std', 'yrly_avg', 'yrly_std'])
df['year'] = list(range(2015, 2023))
df['cum_sum_ia'] = avg_T_ia
df['cum_sum_std_ia'] = std_T_ia
df['yrly_sum_ia'] = Avgs_ia
df['yrly_std_ia'] = Stds_ia
df['cum_sum_a'] = avg_T_a
df['cum_sum_std_a'] = std_T_a
df['yrly_sum_a'] = Avgs_a
df['yrly_std_a'] = Stds_a
df['cum_sum_s'] = avg_T_s
df['cum_sum_std_s'] = std_T_s
df['yrly_sum_s'] = Avgs_s
df['yrly_std_s'] = Stds_s
df.to_pickle('expected_penalty_df.pkl', protocol=5)